In [2]:
from opensearchpy import OpenSearch
import json
host = 'localhost'
port = 9200
auth = ('admin','admin')



In [6]:
# Create the client with SSL/TLS enabled, but hostname and certification verification disabled.
client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    # client_cert = client_cert_path,
    # client_key = client_key_path,
    use_ssl = True,
    verify_certs = False,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)

In [9]:
#Do a few checks before we start indexing:
print(client.cat.health())
print(client.cat.indices())

1645298470 19:21:10 docker-cluster yellow 1 1 true 4 4 0 0 1 0 - 80.0%

yellow open security-auditlog-2022.02.19 PGiBvPgHTmyI16bdT8uDpw 1 1 22 0   83kb   83kb
green  open .kibana_92668751_admin_1     jGcsAkCnQjajrpvfz3yHRw 1 0  1 0 53.7kb 53.7kb
green  open .kibana_1                    wf5MjWZ2TI-JNKzwl1NuDw 1 0  3 7 36.2kb 36.2kb
green  open .opendistro_security         rCemxg5jRBiKHSzJSdR_Fg 1 0  9 0 59.9kb 59.9kb



In [10]:
# If you still have your documents from the Dev Tools test, we should be able to check them here:
print(client.cat.count("searchml_test", params={"v":"true"}))

epoch      timestamp count
1645301091 20:04:51  4

